In [ ]:
# Install the necessary packages
!pip install openai pinecone-client ipywidgets

import openai
import pinecone
import ipywidgets as widgets
from IPython.display import display


In [ ]:
# Set up OpenAI API key
openai.api_key = "openai-api-key"

# Function to generate a response from OpenAI's GPT model
def generate_response(prompt):
    try:
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=150
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return str(e)


In [ ]:
# Initialize Pinecone
pinecone.init(api_key="your-pinecone-api-key", environment='us-west1-gcp')

# Function to connect to a Pinecone index
def connect_to_index(index_name):
    if index_name not in pinecone.list_indexes():
        pinecone.create_index(index_name, dimension=1536, metric='cosine')  # Dimension should match your vector size
    return pinecone.Index(index_name)

# Connect to Pinecone index (or create if it doesn't exist)
index = connect_to_index("your-index-name")  # Replace with your index name


In [ ]:
def embed_text_with_openai(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response['data'][0]['embedding']

In [ ]:
def rag_answer(question):
    # Convert question to vector using OpenAI's embedding model
    question_vector = embed_text_with_openai(question)

    # Query Pinecone with this vector
    top_k_results = index.query([question_vector], top_k=3)

    # Extract retrieved contexts
    retrieved_contexts = [result['context'] for result in top_k_results['matches']]

    # Formulate the prompt for OpenAI
    prompt = f"Question: {question}\nContext: {' '.join(retrieved_contexts)}\nAnswer:"

    # Generate response using OpenAI
    response = generate_response(prompt)
    return response


In [4]:
# Interactive Widgets for asking questions
question_input = widgets.Text(
    value='',
    placeholder='Type your question here',
    description='Question:',
    disabled=False
)

def on_ask_button_clicked(b):
    # Display the answer
    answer = rag_answer(question_input.value)
    print("Answer:", answer)

ask_button = widgets.Button(description="Ask")
ask_button.on_click(on_ask_button_clicked)

display(question_input, ask_button)


Text(value='', description='Question:', placeholder='Type your question here')

Button(description='Ask', style=ButtonStyle())

NameError: name 'rag_answer' is not defined